In [59]:
import pandas as pd
import numpy as np
from datetime import timedelta as td
from datetime import datetime as dt
import xlsxwriter as xl

g = ['US','UK','EC', 'CA','IT','FR','GE','SZ','SP']
s = ['DE','SW','NO']
a = ['AU','NZ']
c = ['HU','CZ','NO']
z = ['SA','IS']
l = ['MX','BZ','CO','CL']
w = g + s + a + c + z + l

day_label = {0: 'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday'}

In [60]:
def country_fix(x):
    if x == 'EC':
        return 'EU'
    elif x == 'UK':
        return 'GB'
    else:
        return x

def no(x):
    if type(x) != float:
        return np.nan
    else:
        return x
    
def percent(x):
    if x != '':
        if type(x) == float and abs(x)<1:
            return str(round(x*100,2))+'% exp'
        else:
            return str(x) +' exp' 
    else:
        return x

#def insert_exp(a,b):
#    if b:
#        return str(a)+' exp'
#    else:
#        return ''

In [71]:
dfs = pd.read_excel('C:\\BLP\\data\\calendar.xlsx')
dfs = dfs[dfs['Unnamed: 1'].isin(w)].copy(deep = True)
dfs.fillna('',inplace = True)

no_time = [i for i in dfs.index if len(dfs.at[i,'Date Time'].split()) < 2]
dfs.drop(labels=no_time, axis = 0, inplace=True)
double_date = [i for i in dfs.index if ':' not in dfs.at[i,'Date Time'].split()[1]]
dfs.drop(labels=double_date, axis = 0, inplace=True)
days_of_week = list(set([dt.strptime(i.split()[0],'%m/%d/%Y') for i in dfs['Date Time']]))
dfs['grouper'] = [dt.strptime(i.split()[0],'%m/%d/%Y') for i in dfs['Date Time']]

dfs.replace('--','',inplace = True)
dfs.reset_index(inplace = True,drop = True)

In [85]:
for d in days_of_week:
    
    day = day_label[d.weekday()]
    df = dfs[dfs['grouper'] == d].copy()
    
    new_df = pd.DataFrame({'a': [country_fix(i) for i in df['Unnamed: 1'] ],
                           'b': [i.split()[1] for i in df['Date Time']],
                           'c': df['Event'],
                           'd':  [percent(i) for i in df['Survey']]})
    

    
    h = open(f"C:\\BLP\\data\\news{d.strftime('%Y_%m_%d')}_{day}.txt","w")
    t = [f'***{day.upper()}***\n\n','***Economic Releases***\n\n','***Speakers***\n\n','***Supply***\n\n']
    L = ['{'+new_df.a[i]+'} '+ new_df.b[i] +', '+ new_df.c[i] +' '+new_df.d[i]+'\n'  for i in new_df.index ]
    h.writelines(t+L)
    h.close()
    